# BERT Model Fine-Tuning

### Load packages

In [1]:
import torch
from transformers import BertTokenizerFast, BertModel, BertForSequenceClassification
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm

In [2]:
from google.colab import files
uploaded = files.upload()

Saving imdb_test.csv to imdb_test.csv
Saving imdb_train.csv to imdb_train.csv


### Loading Datasets

In [3]:
import pandas as pd
train_df = pd.read_csv("imdb_train.csv")
test_df = pd.read_csv("imdb_test.csv")

### Tokenize IMDb dataset

In [4]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
def tokenize_data(texts, labels, max_length=512):
    return tokenizer(texts.tolist(),
                     padding="max_length",
                     truncation=True,
                     max_length=max_length,
                     return_tensors="pt"), labels

In [6]:
train_encodings, train_labels = tokenize_data(train_df["text"], train_df["label"])
test_encodings, test_labels = tokenize_data(test_df["text"], test_df["label"])

### Pytorch Dataset

In [7]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encondings, labels):
        self.encodings = encondings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

In [8]:
train_dataset = IMDbDataset(train_encodings, train_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

### Dataloaders for Batching

In [9]:
batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Fine-Tuning BERT

In [10]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2) #for binary classification: pos & neg
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)

num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

GPU Available: True
GPU Name: Tesla T4


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [14]:
epochs = 3
for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)

    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        # Update progress bar
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())


Epoch 3: 100%|██████████| 3125/3125 [39:52<00:00,  1.31it/s, loss=0.00125]


In [15]:
model.save_pretrained("bert-imdb-model")
tokenizer.save_pretrained("bert-imdb-tokenizer")

('bert-imdb-tokenizer/tokenizer_config.json',
 'bert-imdb-tokenizer/special_tokens_map.json',
 'bert-imdb-tokenizer/vocab.txt',
 'bert-imdb-tokenizer/added_tokens.json',
 'bert-imdb-tokenizer/tokenizer.json')

In [16]:
!ls -l bert-imdb-tokenizer

total 932
-rw-r--r-- 1 root root    125 Mar 10 17:08 special_tokens_map.json
-rw-r--r-- 1 root root   1221 Mar 10 17:08 tokenizer_config.json
-rw-r--r-- 1 root root 711661 Mar 10 17:08 tokenizer.json
-rw-r--r-- 1 root root 231508 Mar 10 17:08 vocab.txt


In [17]:
import shutil
shutil.make_archive("bert-imdb-tokenizer", 'zip', "bert-imdb-tokenizer")

'/content/bert-imdb-tokenizer.zip'

In [18]:
files.download("bert-imdb-tokenizer.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>